In [ ]:
pip install bs4

In [ ]:
pip install html5lib

In [ ]:
from bs4 import BeautifulSoup
import time
import requests
import re
import lxml

In [ ]:

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}
#driver = webdriver.Chrome()
#driver.get(URL)
#time.sleep(3)
#html = driver.page_source
#soup = BeautifulSoup(html, 'html.parser')
#r = requests.get(url=URL, headers=headers)
#driver.switch_to.frame("sp_message_iframe_764224") 
#element = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[3]/div[2]/button').click()

In [ ]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
headers = {'User-Agent': USER_AGENT}

In [ ]:
league_codes = ["AN1L", "AR1N", "AUS1", "A1", "BGD1", "BE1", "BO1A", "BRA1", "CDN1", "CLPD", "CSL", "COLP", "COL1", "CRPD", "PDV1", "KR1", "TS1",
"DK1", 
"EL1A", 
"EL1S", 
"SL1A", 
"GB1", 
"GB2", 
"GB3", 
"GB4", 
"FIJ1", 
"FR1", 
"FR2", 
"L1", 
"L2", 
"L3", 
"GHPL", 
"GR1", 
"GU1A", 
"GU1C", 
"HGKG", 
"UNG1", 
"IND1", 
"IN1L", 
"IRN1", 
"IR1", 
"ISR1", 
"IT1", 
"IT2", 
"JPL1", 
"JAP1", 
"RSK1", 
"KG1L", 
"MYS1", 
"MEXA", 
"MARP", 
"MO1L", 
"MYA1", 
"NPL1", 
"NL1", 
"NCL1", 
"NNL1", 
"NO1", 
"PN1A", 
"PN1C", 
"PR1A", 
"PR1C", 
"TDeA", 
"TDeC", 
"PFL1", 
"PL1", 
"PO1", 
"QSL", 
"RO1", 
"RU1", 
"SA1", 
"SC1", 
"SER1", 
"SIN1", 
"SFA1", 
"ES1", 
"ES2", 
"SE1", 
"C1", 
"TAD1", 
"THA1", 
"TUN1", 
"TUSC", 
"TR1", 
"UKR1", 
"UAE1", 
"MLS1", 
"URU1", 
"UZ1", 
"VZ1L", 
"VIE1"]
list_seasons = ["2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]

In [ ]:
url_temp = "https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/{league}/plus/?saison_id={season}"


In [ ]:
list_urls = []
for season in list_seasons:
    for league in league_codes:
        url = url_temp.format(league=league, season=season)
        print(url)
        r = requests.get(url=url, headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find("table", class_='items')

        if table is not None:
            tbody = table.find("tbody")
            rows = tbody.find_all("tr")
            for row in rows:
                team_links = row.select('a')[0]['href']
                list_urls.append("https://www.transfermarkt.co.uk" + team_links)

        else:
            print('No teams found')

In [ ]:
len(list_urls)

In [ ]:
list_player_url = set()
for link in list_urls:
    print(link)
    r = requests.get(url=link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find("table", class_='items')

    if table is not None:
        tbody = table.find("tbody")
        rows = tbody.find_all("tr")
        for row in rows:
            cells = row.find_all('td', 'hauptlink')
            if cells:
                player_link = cells[0].find('a')['href']
                list_player_url.add("https://www.transfermarkt.co.uk" + player_link)
    else:
        print("Table with class 'items' not found on the page.")

list_player_url = list(list_player_url)


In [ ]:
len(list_player_url)

In [ ]:
def create_dict(player_name, DOB, POB, age, height, citizenship, position, foot, agent, current_club, joined, contract_expiry):

    player_info = {
        "Player Name": player_name,
        "Date of Birth": DOB,
        "Place of Birth": POB,
        "Age": age,
        "Height": height,
        "Citizenship": citizenship,
        "Position": position,
        "Foot": foot,
        "Agent": agent,
        "Current Club": current_club,
        "Joined": joined,
        "Contract Expiry": contract_expiry
    }
    return player_info




In [ ]:
player_bio_list = []


for player in list_player_url:
    print(player)
    r = requests.get(url=player, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    full_name_label = soup.find('span', string=['Full name:', 'Name in home country:'])

    if full_name_label:
        player_name = full_name_label.find_next('span', class_='info-table__content--bold').text.strip()
        print(player_name)
        dob_age_span = soup.find('span', class_='info-table__content--bold')
        if dob_age_span:
            link = dob_age_span.find("a")
            if link:
                dob_text = link.text.strip()  # Get the text inside the <a> tag
        
        # Separate the date and age if format is "Sep 16, 1998 (26)"
                if '(' in dob_text and ')' in dob_text:
                    dob, age = dob_text.split(' (')
                    age = age.rstrip(')')  # Remove the closing parenthesis from age
                    
                    print("Date of Birth:", dob)
                    print("Age:", age)
                else:
                    print("Date of birth and age format not recognized.")
            else:
                print("Date of birth and age link not found.")
        else:
            print("Date of birth and age span not found.")
        POB_tag = soup.find('span', string='Place of birth:')
        POB = POB_tag.find_next('span', class_='info-table__content--bold').text.strip() if POB_tag else None
        print(POB)
        #age_tag = soup.find('span', string='Age:')
        #age = age_tag.find_next('span', class_='info-table__content--bold').text.strip() if age_tag else None
        #print(age)
        height_tag = soup.find('span', string='Height:')
        height = height_tag.find_next('span', class_='info-table__content--bold').text.strip() if height_tag else None
        print(height)
        citizenship_tag = soup.find('span', string='Citizenship:')
        citizenship = citizenship_tag.find_next('span', class_='info-table__content--bold').text.strip() if citizenship_tag else None
        print(citizenship)
        position_tag = soup.find('span', string='Position:')
        position = position_tag.find_next('span', class_='info-table__content--bold').text.strip() if position_tag else None
        print(position)
        foot_tag = soup.find('span', string='Foot:')
        foot = foot_tag.find_next('span', class_='info-table__content--bold').text.strip() if foot_tag else None
        print(foot)
        agent_tag = soup.find('span', string='Agent:')
        agent = agent_tag.find_next('span', class_='info-table__content--bold').text.strip() if agent_tag else None
        print(agent)
        current_club_tag = soup.find('span', string='Current club:')
        current_club = current_club_tag.find_next('a').text.strip() if current_club_tag else None
        print(current_club)
        joined_tag = soup.find('span', string='Joined:')
        joined = joined_tag.find_next('span', class_='info-table__content--bold').text.strip() if joined_tag else None
        print(joined)
        contract_expiry_tag = soup.find('span', string='Contract expires:')
        contract_expiry = contract_expiry_tag.find_next('span', class_='info-table__content--bold').text.strip() if contract_expiry_tag else None
        print(contract_expiry)

        player_bio_dict = create_dict(player_name, DOB, POB, age, height, citizenship, position, foot, agent, current_club, joined, contract_expiry)
        player_bio_list.append(player_bio_dict)

    

In [39]:

player_bio_list = []

for player in list_player_url:
    print(f"Scraping player URL: {player}")
    r = requests.get(url=player, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Extract Current Club (handling NoneType)
    current_club_tag = soup.find('span', class_='info-table__content info-table__content--bold info-table__content--flex')
    if current_club_tag:
        current_club_link = current_club_tag.find('a', title=True)
        if current_club_link:
            current_club = current_club_link.text.strip()  # Extract the current club name
            print("Current Club:", current_club)
        else:
            print("Current club link not found.")
    else:
        print("Current club not found.")
    
    # Find the player's full name (robust check)
    full_name_label = soup.find('span', string=['Full name:', 'Name in home country:'])
    if full_name_label:
        player_name = full_name_label.find_next('span', class_='info-table__content--bold').text.strip() if full_name_label else None
        print("Name:", player_name)
    else:
        print("Full name not found.")

    # Extract DOB and Age (robust check)
    dob_text = None
    dob_age_span = soup.find_all('span', class_='info-table__content--bold')
    for span in dob_age_span:
        if "(" in span.text and ")" in span.text:  # Look for format containing both DOB and Age in parentheses
            dob_text = span.text.strip()
            break

    if dob_text:
        if '(' in dob_text and ')' in dob_text:
            dob, age = dob_text.split(' (')
            dob = dob.strip()            # Format Date of Birth
            age = age.rstrip(')')         # Format Age, remove closing parenthesis
            print("Date of Birth:", dob)
            print("Age:", age)
        else:
            print("Date of birth and age format not recognized.")
    else:
        print("Date of birth and age span not found.")

    # Place of Birth (robust check)
    POB_tag = soup.find('span', string='Place of birth:')
    if POB_tag:
        POB = POB_tag.find_next('span', class_='info-table__content--bold').text.strip() if POB_tag else None
        print("Place of Birth:", POB)
    else:
        print("Place of Birth not found.")

    # Height
    height_tag = soup.find('span', string='Height:')
    if height_tag:
        height = height_tag.find_next('span', class_='info-table__content--bold').text.strip()
        print("Height:", height)
    else:
        print("Height not found.")

    # Citizenship
    citizenship_tag = soup.find('span', string='Citizenship:')
    if citizenship_tag:
        citizenship = citizenship_tag.find_next('span', class_='info-table__content--bold').text.strip()
        print("Citizenship:", citizenship)
    else:
        print("Citizenship not found.")

    # Position
    position_tag = soup.find('span', string='Position:')
    if position_tag:
        position = position_tag.find_next('span', class_='info-table__content--bold').text.strip()
        print("Position:", position)
    else:
        print("Position not found.")

    # Preferred Foot
    foot_tag = soup.find('span', string='Foot:')
    if foot_tag:
        foot = foot_tag.find_next('span', class_='info-table__content--bold').text.strip()
        print("Foot:", foot)
    else:
        print("Foot not found.")

    # Agent
    agent_tag = soup.find('span', string='Agent:')
    if agent_tag:
        agent = agent_tag.find_next('span', class_='info-table__content--bold').text.strip()
        print("Agent:", agent)
    else:
        print("Agent not found.")

    # Joined Date
    joined_tag = soup.find('span', string='Joined:')
    if joined_tag:
        joined = joined_tag.find_next('span', class_='info-table__content--bold').text.strip()
        print("Joined:", joined)
    else:
        print("Joined not found.")

    # Contract Expiry
    contract_expiry_tag = soup.find('span', string='Contract expires:')
    if contract_expiry_tag:
        contract_expiry = contract_expiry_tag.find_next('span', class_='info-table__content--bold').text.strip()
        print("Contract Expires:", contract_expiry)
    else:
        print("Contract Expiry not found.")

    # Compile player bio into a dictionary
    player_bio_dict = {
        "Name": player_name,
        "Current Club": current_club if current_club else "N/A",  # Include current club, default to "N/A"
        "Date of Birth": dob if dob else "N/A",
        "Age": age if age else "N/A",
        "Place of Birth": POB if POB else "N/A",
        "Height": height if height else "N/A",
        "Citizenship": citizenship if citizenship else "N/A",
        "Position": position if position else "N/A",
        "Foot": foot if foot else "N/A",
        "Agent": agent if agent else "N/A",
        "Joined": joined if joined else "N/A",
        "Contract Expires": contract_expiry if contract_expiry else "N/A"
    }

    player_bio_list.append(player_bio_dict)
    print("Player bio added:", player_bio_dict)
    print("----------")

# Output the full player bio list
print(player_bio_list)


Scraping player URL: https://www.transfermarkt.co.uk/markus-kuster/profil/spieler/123147
Current club link not found.
Full name not found.
Date of Birth: Feb 22, 1994
Age: 30
Place of Birth: Wien
Height: 1,95 m
Citizenship: Austria
Position: Goalkeeper
Foot: right
Agent not found.
Joined: Jan 1, 2023
Contract Expires: Jun 30, 2025
Player bio added: {'Name': None, 'Current Club': 'N/A', 'Date of Birth': 'Feb 22, 1994', 'Age': '30', 'Place of Birth': 'Wien', 'Height': '1,95\xa0m', 'Citizenship': 'Austria', 'Position': 'Goalkeeper', 'Foot': 'right', 'Agent': 'N/A', 'Joined': 'Jan 1, 2023', 'Contract Expires': 'Jun 30, 2025'}
----------
Scraping player URL: https://www.transfermarkt.co.uk/gabriel-panzo/profil/spieler/1320705
Current Club: Académica Petróleos do Lobito
Full name not found.
Date of Birth: Oct 13, 2002
Age: 22
Place of Birth not found.
Height not found.
Citizenship: Angola
Position: Goalkeeper
Foot not found.
Agent not found.
Joined: -
Contract Expires: -
Player bio added: {'

KeyboardInterrupt: 

In [1]:
print(player_bio_list)

NameError: name 'player_bio_list' is not defined